In [233]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as BS

url = "http://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&tab=main"
# 브라우저 세션 자동화
driver = webdriver.Chrome("/Users/yoon/.selenium-drivers/chromedriver")

# 주어진 url로 이동한다
driver.get(url)

# 현재 페이지 소스를 가져온다
html = driver.page_source

# soup object: parsed document
soup = BS(html, "html.parser")

In [240]:
def get_prod_items(prodduct):
    prod_dicts = {
        "상품명": [],
        "상세_스펙": [],
        "가격": []
    }

    for prod in products:
        try:
            prod_dicts["상품명"].append(prod.select("p.prod_name > a")[0].text.strip())
        except:
            prod_dicts["상품명"].append("")
        
        try:
            prod_dicts["상세_스펙"].append(prod.select("div.spec_list")[0].text.strip())
        except:
            prod_dicts["상세_스펙"].append("")
            
        try:
            prod_dicts["가격"].append(prod.select("p.price_sect > a > strong")[0].text.strip())
        except:
            prod_dicts["가격"].append(0)
        
    return prod_dicts

In [241]:
products = soup.select("div.main_prodlist.main_prodlist_list > ul[class='product_list'] > li.prod_item")

In [242]:
def get_search_page_url(keyword, page):
    return ("http://search.danawa.com/dsearch.php"
            + "?query="
            + keyword
            + "&volumeType=allvs"
            + "&page="
            + str(page)
            + "&limit=40"
            + "&sort=saveDESC"
            + "&list=list"
            + "&boost=true"
            + "&addDelivery=N"
            + "&tab=goods")

## 진행상황 출력하기
* tqdm module

In [243]:
from time import sleep
from tqdm import tqdm_notebook

driver = webdriver.Chrome("/Users/yoon/.selenium-drivers/chromedriver")
keyword = "무선청소기"
total_page = 10
products_df = pd.DataFrame(columns=['상품명', '상세_스펙', '가격'])

# 진행 상황 출력
for page in tqdm_notebook(range(1, total_page + 1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    sleep(2)
    
    html = driver.page_source
    soup = BS(html, "html.parser")
    products = soup.select("div.main_prodlist.main_prodlist_list > ul[class='product_list'] > li.prod_item")
    df = pd.DataFrame(get_prod_items(products))
    products_df = products_df.append(df, ignore_index=True)


<ipython-input-243-48e257b279f3>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1, total_page + 1)):


In [244]:
products_df.loc[products_df['가격'] == '일시품절']

,상품명,상세_스펙,가격
119,LG전자 코드제로 A9 A930,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 헤파필터 / 2중터보싸이클론 / 5단...,일시품절
359,트윈버드 TC-E123SBK,핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 싸이클론 / 흡입력: 70W / 브러...,일시품절


In [245]:
products_df.drop(products_df[products_df['가격'] == '일시품절'].index, inplace=True)

In [246]:
products_df['가격'] = products_df['가격'].str.replace(',','').astype(int)

In [249]:
products_df.sort_values(by='가격', inplace=True)
products_df.reset_index(drop=True, inplace=True)
products_df.to_excel("danawa_crawling_result.xlsx", index=False)

In [250]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     398 non-null    object
 1   상세_스펙   398 non-null    object
 2   가격      398 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.5+ KB


## 데이터 전처리

In [251]:
data = pd.read_excel("danawa_crawling_result.xlsx")

In [252]:
data.head()

,상품명,상세_스펙,가격
0,USB 미니청소기,핸디/스틱청소기 / 핸디형 / 무선형 / 색상:블랙 / USB충전식,5000
1,유니맥스 UVC-1688,핸디/스틱청소기 / 핸디형 / 무선형 / 전압: 6V / 싸이클론 / 충전시간: 1...,16840
2,ZHENHUI 미니 진공 차량용 무선 청소기 (해외구매),"차량용청소기 / 무선 / 흡입력: 6,000Pa / 최대출력: 120W / 헤파필터...",18710
3,카스 오토스핀 HS-700,핸디/스틱청소기 / 핸디형 / 무선형 / 충전시간: 9시간 / 색상:블루,19400
4,듀플렉스 DP-485VC,핸디/스틱청소기 / 핸디형 / 무선형 / 전압: 4.8V / 2단계여과 / 브러쉬:...,21070


In [261]:
for title in data['상품명']:
    print(title.split(" ", 1))

['USB', '미니청소기']
['유니맥스', 'UVC-1688']
['ZHENHUI', '미니 진공 차량용 무선 청소기 (해외구매)']
['카스', '오토스핀 HS-700']
['듀플렉스', 'DP-485VC']
['홍진테크', 'AT-H12V']
['LG전자', 'V-S300C']
['차량용', '무선 건습식 청소기 (해외구매)']
['루베크', 'CM-268']
['마이프랜드', 'MTU-188C']
['대웅모닝컴', 'DWVC-AC800KP']
['듀플렉스', 'DP-480VC']
['샤오미', 'SHUNZAO 차량용 무선청소기 2세대 Z1 (해외구매)']
['미라스', 'FX-178VC']
['신일산업', 'SVC-680G']
['비바', '2017-SC01']
['흥진테크', 'SGC-399AA']
['CNON', ' 무선 핸디 청소기 XCBYD (해외구매)']
['샤오미', '이지에 YE-01']
['신일산업', 'SVC-D900RK']
['마끼다', 'CL100DZ']
['유니맥스', 'UVC-1685A']
['머레이', '핸디형 무선청소기 HK-6030']
['신일산업', 'SVC-S500YT']
['신일산업', 'SVC-650SG']
['삼성전자', 'VC-H20']
['삼성전자', 'VC-H22']
['SUNAEssi', 'CZ-S567WM']
['신일산업', 'SVC-4807SG']
['마끼다', 'DCL180Z']
['신일산업', 'SVC-SHS500']
['신일산업', 'SVC-1316R']
['디엔', '무선 물걸레 청소기']
['이나프', 'ILVH-2000']
['보만', 'VC7210']
['라이프썸', 'LFS-HA04']
['보쉬', 'GAS10.8V-LI']
['신일산업', 'SVC-D950GK']
['idam', '701B1']
['신일산업', 'SVC-S4800SJ']
['마이크로스톤', '블랙샤크']
['무궁화전자', '바로바로 MC-B250']
['보만', 'VC7211']
['서주글로벌', '더존 무선 핸디 자동차

In [257]:
company_list, product_list = [], []

for product in data['상품명']:
    product = product.split(" ", 1)
    company_list.append(product[0])
    product_list.append(product[1])

In [320]:
import re

category_list = []
usage_time_list = []
suction_power_list = []

for spec in data['상세_스펙']:
    spec = spec.split(" / ")
    category_list.append(spec[0].strip())
    usage_time, suction_power = "", ""
    
    for elem in spec:
        if re.search("사용시간", elem):
            usage_time = elem.split(":")[1].strip()
        elif re.search("흡입력", elem):
            suction_power = elem.split(":")[1].strip()
            
    usage_time_list.append(usage_time)
    suction_power_list.append(suction_power)

In [290]:
print(len(category_list), len(usage_time_list), len(suction_power_list))

398 398 398


In [340]:
temp_dict = {
    "카테고리": category_list,
    "사용시간": usage_time_list,
    "흡입력": suction_power_list
}
temp = pd.DataFrame(temp_dict)


In [344]:
temp["사용시간"].unique()

array(['', '20~30분', '20분', '15~18분', '8분', '30분', '15분', '60분', '2시간',
       '12분', '9분', '5~10분', '10분', '5분~10분', '25분', '10~15분', '7분',
       '16분', '1시간10분', '12~15분', '7~10분', '11분', '1시간40분', '40분', '사용시간',
       '50분', '10~12분', '5시간', '35분', '14분', '42분', '23분', '80분', '24분',
       '17분', '1시간', '45분', '22분', '13분29초', '18분', '5.0Ah', '1시간20분',
       '물걸레', '48분', '12~30분', '1시간30분', '55분', '1시간5분', '1시간6분', '53분',
       '65분', '40분(강), 70분(약)', '3시간', '120분', '1시간~1시간30분'], dtype=object)

In [348]:
temp[temp["사용시간"] == "사용시간"]

,카테고리,사용시간,흡입력
64,핸디/스틱청소기,사용시간,


In [413]:
def convert_hours_to_minutes(t):
    total_time = 0
    

    if "사용" in t or "Ah" in t or "" in t:
        return 0
    elif "시간" in t:
        print(t)
        if "~" in t:
            #print(t.split("~")[0].split("시간")[0], t.split("~")[1].split("시간")[0], t.split("시간")[2].split("분")[0])
            total_time += (int(t.split("~")[0].split("시간")[0]) + int(t.split("~")[1].split("시간")[0])) * 60 + int(t.split("시간")[2].split("분")[0])
        else:
            total_time += int(t.split("시간")[0]) * 60
            if "분" in t:
                total_time += int(t.split("시간")[1].split("분")[0])
    else:
        if "~" in t:
            total_time += (int(t.split("~")[0]) + int(t.split("~")[1].split("분")[0])) // 2
        else:
            total_time += int(t.split("분")[0])
        
    return total_time


times = ["40분", "4분", "1시간", "3시간30분", "7~10분", "5.0Ah", "사용시간", "" , "1시간~1시간30분"]

for t in times:
    time_value = convert_hours_to_minutes(t)
    #print(t, '=', time_value)

In [379]:
"1시간~1시간30분".split("~")[0].split("시간")[0] + "1시간~1시간30분".split("~")[1].split("시간")[0] 

'11'

In [388]:
"1시간~1시간30분".split("시간")[2].split("분")[0]

'30'

In [394]:
"1시간~1시간30분".split("~")[1].split("시간")[0]

'1'